In [1]:
%load_ext autoreload
%autoreload
%matplotlib inline

from news_crawler import NewsCrawler, select_urls
from IPython.display import display
from IPython.core.debugger import set_trace

In [2]:
crawler = NewsCrawler()

# 1. URL을 모은다

In [3]:
# 제거고려: nextbigfuture, nbcnews, joongang, chosun

In [28]:
collect_result = crawler.collect(); display(*collect_result)

100.00% (100.64 seconds): Collecting URLs... yahoo               

100.00% (1.78 seconds): Selecting URLs... cryptonews          

,collected,collected_unique,selected
all,26122,26059,363
yahoo,393,393,60
reuters,1149,1149,47
investing.com,930,930,38
dailymail,2896,2896,32
...,...,...,...
blastingnews,122,122,0
npr,896,896,0
nationalreview,735,734,0
msnbc,135,104,0


,url,pubs,actual_pub
0,https://politico.com/news/2019/10/17/pelosi-de...,"politico, realclearpolitics",politico
1,https://politico.com/news/2019/10/16/elizabeth...,"politico, realclearpolitics",politico
2,https://politico.com/news/2019/10/17/trump-syr...,"politico, realclearpolitics",politico
3,https://politico.com/magazine/story/2019/10/15...,"politico, realclearpolitics",politico
4,https://politico.com/news/2019/10/15/trump-con...,"politico, realclearpolitics",politico
...,...,...,...
58,https://msnbc.com/msnbc/watch/mulvaney-says-tr...,"msnbc, nbcnews",msnbc
59,https://msnbc.com/ali-velshi/watch/romney-aban...,"msnbc, nbcnews",msnbc
60,https://nbcnews.com/politics/politics-news/pen...,"msnbc, nbcnews",nbcnews
61,https://nbcnews.com/news/latino/house-holds-it...,"msnbc, nbcnews",nbcnews


# 2. 해당 URL의 뉴스를 다운로드한다

In [29]:
download_result = crawler.download(); download_result

25.62% (23.10 seconds): Downloading... businessinsider     

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


100.00% (243.86 seconds): Downloading... yahoo               

,downloaded,trashed,total
all,235,128,363
yahoo,48,12,60
reuters,25,22,47
investing.com,17,21,38
dailymail,29,3,32
...,...,...,...
blastingnews,0,0,0
npr,0,0,0
nationalreview,0,0,0
msnbc,0,0,0
